# The Power of Retrieval Augmented Generation: a comparison between RAGs and Fine-Tuned LLMs
Here we explore how to make a LLM give accurate answers about the lastest news of OpenAI without being fine-tuned on such data but instead, use the power of RAGs

### Import Libraries

In [1]:
from langchain.docstore.document import Document
from langchain.document_loaders import HuggingFaceDatasetLoader

from encoder.encoder import Encoder
from generator.generator import Generator
from retriever.vector_db import VectorDatabase

### Define Global Variables

In [2]:
TEMPLATE = """
Use the following pieces of context to answer the question at the end taking in consideration the dates. 
{context}
Question: {question}
Answer:
"""

QUERY = "What happened to the CEO of OpenAI?"

### Load Dataset and preprocess it

In [3]:
# Get some open ai news to add to the final dataset
openai_news = [
    "2023-11-22 - Sam Altman returns to OpenAl as CEO with a new initial board of Bret Taylor (Chair), Larry Summers, and Adam D'Angelo.",
    "2023-11-21 - Ilya and the board's decision to fire Sam from OpenAI caught everyone off guard, with no prior information shared.",
    "2023-11-21 - In a swift response, Sam was welcomed into Microsoft by Satya Nadella himself.",
    "2023-11-21 - Meanwhile, a staggering 500+ OpenAI employees made a bold move, confronting the board with a letter: either step down or they will defect to Sam's new team at Microsoft.",
    "2023-11-21 - In a jaw-dropping twist, Ilya, integral to Sam's firing, also put his name on that very letter. Talk about an unexpected turn of events!",
    "2023-11-20 - BREAKING: Sam Altman and Greg Brockman Join Microsoft, Emmett Shear Appointed CEO of OpenAI",
    "2023-11-20 - Microsoft CEO Satya Nadella announced a major shift in their partnership with OpenAI. Sam Altman and Greg Brockman, key figures at OpenAI, are now joining Microsoft to lead a new AI research team. This move marks a significant collaboration and potential for AI advancements. Additionally, Emmett Shear, former CEO of Twitch, has been appointed as the new CEO of OpenAI, signaling a new chapter in AI leadership and innovation.",
    "2023-11-20 - Leadership Shakeup at OpenAI - Sam Altman Steps Down!",
    "2023-11-20 - Just a few days after presenting at OpenAI's DevDay, CEO Sam Altman has unexpectedly departed from the company, and Mira Murati, CTO of the company, steps in as Interim CEO. This is a huge surprise and speaks volumes about the dynamic shifts in tech leadership today.",
    """2023-11-20 - What's Happening at OpenAI?
    - Sam Altman, the face of OpenAI, is leaving not just the CEO role but also the board of directors.
    - Mira Murati, an integral part of OpenAI's journey and a tech visionary, is taking the helm as interim CEO.
    - The board is now on a quest to find a permanent successor.""",
    "2023-11-20 - The transition raises questions about the future direction of OpenAI, especially after the board's statement about losing confidence in Altman's leadership.",
    """2023-11-20 - With a board consisting of AI and tech experts like Ilya Sutskever, Adam D’Angelo, Tasha McCauley, and Helen Toner, OpenAI is poised to continue its mission. Can they do it without Sam?
    - Greg Brockman, stepping down as chairman, will still play a crucial role, reporting to the new CEO."""
]

In [5]:
# load dataset with some news
loader = HuggingFaceDatasetLoader("cnn_dailymail", "highlights", name='3.0.0')
docs = loader.load()[:10000] # get a sample of news

# add openai news to our list of docs
docs.extend([
    Document(page_content=x) for x in openai_news
])

/Users/rafael/miniconda3/envs/zaai/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Generating test split: 100%|██████████| 11490/11490 [00:33<00:00, 341.29 examples/s] 


### Initiate our RAG modules

In [6]:
# initiate our classes for the Encoder, Retriever and Generator
encoder = Encoder()
faiss_db = VectorDatabase()
generator = Generator(TEMPLATE)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/rafael/Documents/large-language-models/rag/model/nous-hermes-llama-2-7b.Q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn

### Create and store passages in a vector database

In [7]:
# Create passages and store them in a vector DB
passages = faiss_db.create_passages_from_documents(docs)
faiss_db.store_passages_db(passages, encoder.encoder)

### Retrieve most similar document

In [13]:
# retrive most similar document to our query
context = faiss_db.retrieve_most_similar_document(QUERY, k=8)
print(context)

2023-11-20 - What's Happening at OpenAI?
    - Sam Altman, the face of OpenAI, is leaving not just the CEO role but also the board of directors.
    - Mira Murati, an integral part of OpenAI's journey and a tech visionary, is taking the helm as interim CEO.
    - The board is now on a quest to find a permanent successor.
2023-11-20 - Microsoft CEO Satya Nadella announced a major shift in their partnership with OpenAI. Sam Altman and Greg Brockman, key figures at OpenAI, are now joining Microsoft to lead a new AI research team. This move marks a significant collaboration and potential for AI advancements. Additionally, Emmett Shear, former CEO of Twitch, has been appointed as the new CEO of OpenAI, signaling a new chapter in AI leadership and innovation.
2023-11-20 - With a board consisting of AI and tech experts like Ilya Sutskever, Adam D’Angelo, Tasha McCauley, and Helen Toner, OpenAI is poised to continue its mission. Can they do it without Sam?
    - Greg Brockman, stepping down as

In [16]:
# RAG LLama
print(generator.get_answer(context, QUERY))

Llama.generate: prefix-match hit


Sam Altman stepped down from his role as CEO of OpenAI on November 20th, 2023. He was replaced by Mira Murati who assumed the position of interim CEO. However, Sam Altman returned to the company a few days later as CEO with a new initial board consisting of Bret Taylor (Chair), Larry Summers and Adam D'Angelo.



llama_print_timings:        load time =    9494.53 ms
llama_print_timings:      sample time =      22.20 ms /    84 runs   (    0.26 ms per token,  3783.78 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   18738.01 ms /    84 runs   (  223.07 ms per token,     4.48 tokens per second)
llama_print_timings:       total time =   18995.42 ms


In [26]:
# Base LLama
print(generator.get_answer('', QUERY))


Llama.generate: prefix-match hit
OpenAI is a non-profit artificial intelligence research organization based in San Francisco, California, that focuses on developing friendly AI and ensuring its safe use. The CEO of OpenAI is Sam Altman, who has been at the helm since 2010 when he co-founded the company with Elon Musk, Greg Brockman, Ilya Sutskever, and Vince Conitzer. In terms of recent news, there haven’t been any major developments regarding the CEO or OpenAI specifically, but it is known that OpenAI has recently expanded its team by hiring an additional 15 employees to work on various projects related to AI, including the development of new models and tools for machine learning

llama_print_timings:        load time =   10216.91 ms
llama_print_timings:      sample time =      41.10 ms /   125 runs   (    0.33 ms per token,  3041.51 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_